In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, StringVar, ttk
from PIL import Image, ImageTk  # Para trabajar con imágenes
import psycopg2
import openpyxl
import os

# 1. Función para conectar a la base de datos
def conectar_bd():
    try:
        conn = psycopg2.connect(
            dbname="inventario",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432"
        )
        print("Conexión exitosa a la base de datos")
        return conn
    except Exception as e:
        print(f"Error de conexión: {e}")
        messagebox.showerror("Error de Conexión", f"No se pudo conectar a la base de datos: {e}")
        return None

# 2. Función para limpiar los campos del formulario
def limpiar_campos():
    centro_operacion_entry.delete(0, tk.END)
    secuencia_entry.delete(0, tk.END)
    codigo_barras_entry.delete(0, tk.END)
    serie_entry.delete(0, tk.END)
    descripcion_elemento_entry.delete(0, tk.END)
    tipo_parte_entry.delete(0, tk.END)
    tipo_almacenamiento_entry.delete(0, tk.END)
    fecha_ingreso_entry.delete(0, tk.END)
    cedula_resp_entry.set('')
    responsable_inventario_entry.set('')
    ubicacion_var.set("")
    contrato_oc_entry.delete(0, tk.END)
    num_comprobante_entry.delete(0, tk.END)
    proveedor_entry.delete(0, tk.END)

    # Limpiar imágenes
    for label in [imagen_label1, imagen_label2, imagen_label3, imagen_label4]:
        label.config(image='', text='Imagen no seleccionada')

# 3. Función para seleccionar imágenes
def seleccionar_imagen(label):
    filename = filedialog.askopenfilename(
        title="Seleccionar Imagen",
        filetypes=[("Archivos de imagen", "*.jpg *.jpeg *.png *.bmp")]
    )
    if filename:
        img = Image.open(filename)
        img.thumbnail((150, 150))  # Redimensionar la imagen para la vista previa
        img = ImageTk.PhotoImage(img)
        label.config(image=img, text='')
        label.image = img  # Para evitar que el recolector de basura elimine la imagen
        label.path = filename  # Almacenar la ruta de la imagen en el label

# 4. Función para insertar un nuevo registro con imágenes
def insertar_nuevo_registro():
    centro_operacion = centro_operacion_entry.get()
    secuencia = secuencia_entry.get()
    codigo_barras = codigo_barras_entry.get()
    serie = serie_entry.get()
    descripcion = descripcion_elemento_entry.get()
    tipo_parte = tipo_parte_entry.get()
    tipo_almacenamiento = tipo_almacenamiento_entry.get()
    fecha_ingreso = fecha_ingreso_entry.get()
    cedula_resp = cedula_resp_entry.get()
    responsable_inventario = responsable_inventario_entry.get()
    ubicacion = ubicacion_var.get()
    contrato_oc = contrato_oc_entry.get()
    num_comprobante = num_comprobante_entry.get()
    proveedor = proveedor_entry.get()

    # Obtener las rutas de las imágenes seleccionadas
    imagenes = [label.path if hasattr(label, 'path') else None for label in [imagen_label1, imagen_label2, imagen_label3, imagen_label4]]

    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            # Insertar el registro en la base de datos (guardando las rutas de las imágenes)
            cursor.execute("""
                INSERT INTO inv_cgr_verificar (centro_operacion, secuencia, codigo_barras, serie, descripcion_elemento, 
                                               tipo_parte, tipo_almacenamiento, fecha_ingreso, cedula_resp, responsable_inventario, 
                                               ubicacion, contrato_oc, num_comprobante, proveedor, imagen1, imagen2, imagen3, imagen4, verificado)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, FALSE);
            """, (centro_operacion, secuencia, codigo_barras, serie, descripcion, tipo_parte, tipo_almacenamiento, fecha_ingreso, 
                  cedula_resp, responsable_inventario, ubicacion, contrato_oc, num_comprobante, proveedor,
                  imagenes[0], imagenes[1], imagenes[2], imagenes[3]))
            conn.commit()
            messagebox.showinfo("Nuevo Registro", "Se ha insertado un nuevo registro exitosamente.")
        except Exception as e:
            conn.rollback()
            print(f"Error al insertar el nuevo registro: {e}")
        finally:
            cursor.close()
            conn.close()

# 5. Configuración de la ventana principal de Tkinter
root = tk.Tk()
root.title("Verificación Inventario Bienes Devolutivos")

# Crear los frames para dividir la pantalla
frame_left = tk.Frame(root)
frame_left.grid(row=0, column=0, padx=20, pady=20, sticky="n")

# Búsqueda por código de barras
tk.Label(frame_left, text="Buscar por Código de Barras:").grid(row=0, column=0, sticky="w")
codigo_barras_entry = tk.Entry(frame_left, width=50)
codigo_barras_entry.grid(row=0, column=1, padx=5)

# Campos del formulario con ancho de 50
centro_operacion_entry = tk.Entry(frame_left, width=50)
secuencia_entry = tk.Entry(frame_left, width=50)
serie_entry = tk.Entry(frame_left, width=50)
descripcion_elemento_entry = tk.Entry(frame_left, width=50)
tipo_parte_entry = tk.Entry(frame_left, width=50)
tipo_almacenamiento_entry = tk.Entry(frame_left, width=50)
fecha_ingreso_entry = tk.Entry(frame_left, width=50)
cedula_resp_entry = ttk.Combobox(frame_left, textvariable=StringVar(), width=47)
responsable_inventario_entry = ttk.Combobox(frame_left, textvariable=StringVar(), width=47)

# Variables para Combobox
ubicacion_var = StringVar()

# Nuevos campos del formulario
contrato_oc_entry = tk.Entry(frame_left, width=50)
num_comprobante_entry = tk.Entry(frame_left, width=50)
proveedor_entry = tk.Entry(frame_left, width=50)

# Agregamos las etiquetas y campos de entrada
campos = [
    ("Centro Operación", centro_operacion_entry),
    ("Secuencia", secuencia_entry),
    ("Serie", serie_entry),
    ("Descripción Elemento", descripcion_elemento_entry),
    ("Tipo Parte", tipo_parte_entry),
    ("Tipo Almacenamiento", tipo_almacenamiento_entry),
    ("Fecha Ingreso", fecha_ingreso_entry),
    ("Cédula Responsable", cedula_resp_entry),
    ("Responsable Inventario", responsable_inventario_entry),
    ("Contrato OC", contrato_oc_entry),
    ("Num. Comprobante", num_comprobante_entry),
    ("Proveedor", proveedor_entry)
]

for i, (label_text, entry) in enumerate(campos):
    tk.Label(frame_left, text=f"{label_text}:").grid(row=i + 1, column=0, sticky="w")
    entry.grid(row=i + 1, column=1, columnspan=2, padx=5, pady=2)

# Configurar etiquetas de imagenes
imagen_label1 = tk.Label(frame_left, text="Imagen 1")
imagen_label2 = tk.Label(frame_left, text="Imagen 2")
imagen_label3 = tk.Label(frame_left, text="Imagen 3")
imagen_label4 = tk.Label(frame_left, text="Imagen 4")

imagen_label1.grid(row=len(campos) + 1, column=0, padx=5, pady=5)
imagen_label2.grid(row=len(campos) + 1, column=1, padx=5, pady=5)
imagen_label3.grid(row=len(campos) + 2, column=0, padx=5, pady=5)
imagen_label4.grid(row=len(campos) + 2, column=1, padx=5, pady=5)

# Botones para seleccionar imágenes
tk.Button(frame_left, text="Seleccionar Imagen 1", command=lambda: seleccionar_imagen(imagen_label1)).grid(row=len(campos) + 3, column=0)
tk.Button(frame_left, text="Seleccionar Imagen 2", command=lambda: seleccionar_imagen(imagen_label2)).grid(row=len(campos) + 3, column=1)
tk.Button(frame_left, text="Seleccionar Imagen 3", command=lambda: seleccionar_imagen(imagen_label3)).grid(row=len(campos) + 4, column=0)
tk.Button(frame_left, text="Seleccionar Imagen 4", command=lambda: seleccionar_imagen(imagen_label4)).grid(row=len(campos) + 4, column=1)

# Botón para insertar el registro
tk.Button(frame_left, text="Insertar Registro", command=insertar_nuevo_registro).grid(row=len(campos) + 5, column=1, pady=10)

# Ejecutar la aplicación
root.mainloop()
